In [1]:
import numpy as np
import pandas as pd

path='E:/dataset/Kaggle_Santander/cache/'
df_99=pd.read_csv(path+'non_fake_sub_lag_-99999.csv')
df_leak=pd.read_csv(path+'non_fake_sub_lag_29.csv')
df_new=pd.read_csv(path+'blend04.csv')

In [2]:
df_99.head()

,ID,target
0,000137c73,0.0
1,00021489f,0.0
2,0004d7953,-99999.0
3,00056a333,-99999.0
4,00056d8eb,0.0


In [3]:
df_leak.head()

,ID,target
0,000137c73,0.000000e+00
1,00021489f,0.000000e+00
2,0004d7953,1.276252e+06
3,00056a333,7.871320e+06
4,00056d8eb,0.000000e+00


In [4]:
df_new.head()

,ID,target
0,000137c73,1.896260e+06
1,00021489f,1.196232e+06
2,0004d7953,1.392992e+06
3,00056a333,5.715556e+06
4,00056d8eb,1.548354e+06


In [6]:
non_ids=df_99['target'] == -99999
non_ids.head()

0    False
1    False
2     True
3     True
4    False
Name: target, dtype: bool

In [36]:
result_df=df_leak.merge(df_new,on='ID')
result_df.head()

,ID,target_x,target_y
0,000137c73,0.000000e+00,1.896260e+06
1,00021489f,0.000000e+00,1.196232e+06
2,0004d7953,1.276252e+06,1.392992e+06
3,00056a333,7.871320e+06,5.715556e+06
4,00056d8eb,0.000000e+00,1.548354e+06


In [37]:
leak_ids=(df_99['target'] !=-99999) & (df_99['target']!=0)

In [30]:
#按照df_new来存放所有的泄露值,target_y  ，线上0.64分
result_df.loc[non_ids,'target_y']=result_df.loc[non_ids,'target_x']*0.5+result_df.loc[non_ids,'target_y']*0.5
result_df[['ID','target_y']].to_csv(path+'使用063的结果来填充所有泄露行.csv',index=False)
result_df.head()

,ID,target_x,target_y
0,000137c73,0.000000e+00,1.896260e+06
1,00021489f,0.000000e+00,1.196232e+06
2,0004d7953,1.276252e+06,1.334622e+06
3,00056a333,7.871320e+06,6.793438e+06
4,00056d8eb,0.000000e+00,1.548354e+06


In [38]:
#按照df_new来存放所有的泄露值,target_x ，线上0.67分
result_df.loc[non_ids,'target_x']=result_df.loc[non_ids,'target_x']*0.5+result_df.loc[non_ids,'target_y']*0.5
result_df[['ID','target_x']].to_csv(path+'使用069的结果来填充所有泄露行.csv',index=False)
result_df.head()

,ID,target_x,target_y
0,000137c73,0.000000e+00,1.896260e+06
1,00021489f,0.000000e+00,1.196232e+06
2,0004d7953,1.334622e+06,1.392992e+06
3,00056a333,6.793438e+06,5.715556e+06
4,00056d8eb,0.000000e+00,1.548354e+06


In [ ]:
#接下来需要判断，是不是哪些泄露值不是绝对准确，或者是说那些空值也是有一定影响的，
1、如果把0.64得分的文件去掉假行对应的值呢？
    那么如果分数比0.64小，说明假行还是有用的，
    如果比0.64大，同时小于0.67分，那么说明泄露值还是不够准确。